In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,roc_auc_score

### EDA and Data Exploration

In [2]:
df = pd.read_csv('gapminder (1).csv')

In [3]:
df.head()

population  fertility  HIV        CO2  BMI_male      GDP  BMI_female  life  \
0  34811059.0       2.73  0.1   3.328945  24.59620  12314.0    129.9049  75.3   
1  19842251.0       6.43  2.0   1.474353  22.25083   7103.0    130.1247  58.3   
2  40381860.0       2.24  0.5   4.785170  27.50170  14646.0    118.8915  75.5   
3   2975029.0       1.40  0.1   1.804106  25.35542   7383.0    132.8108  72.5   
4  21370348.0       1.96  0.1  18.016313  27.56373  41312.0    117.3755  81.5   

   child_mortality                      Region  
0             29.5  Middle East & North Africa  
1            192.0          Sub-Saharan Africa  
2             15.4                     America  
3             20.0       Europe & Central Asia  
4              5.2         East Asia & Pacific

In [4]:
df.describe() ## Statistical Summary of all your numerical variables

population   fertility         HIV         CO2    BMI_male  \
count  1.390000e+02  139.000000  139.000000  139.000000  139.000000   
mean   3.549977e+07    3.005108    1.915612    4.459874   24.623054   
std    1.095121e+08    1.615354    4.408974    6.268349    2.209368   
min    2.773150e+05    1.280000    0.060000    0.008618   20.397420   
25%    3.752776e+06    1.810000    0.100000    0.496190   22.448135   
50%    9.705130e+06    2.410000    0.400000    2.223796   25.156990   
75%    2.791973e+07    4.095000    1.300000    6.589156   26.497575   
max    1.197070e+09    7.590000   25.900000   48.702062   28.456980   

                 GDP  BMI_female        life  child_mortality  
count     139.000000  139.000000  139.000000       139.000000  
mean    16638.784173  126.701914   69.602878        45.097122  
std     19207.299083    4.471997    9.122189        45.724667  
min       588.000000  117.375500   45.200000         2.700000  
25%      2899.000000  123.232200   62.200000         8.100000  
50%      9938.000000  126.519600   72.000000        24.000000  
75%     23278.500000  130.275900   76.850000        74.200000  
max    126076.000000  135.492000   82.600000       192.000000

In [5]:
df.info() ## Data-type of columns, Number of na values in each column,shape of the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   population       139 non-null    float64
 1   fertility        139 non-null    float64
 2   HIV              139 non-null    float64
 3   CO2              139 non-null    float64
 4   BMI_male         139 non-null    float64
 5   GDP              139 non-null    float64
 6   BMI_female       139 non-null    float64
 7   life             139 non-null    float64
 8   child_mortality  139 non-null    float64
 9   Region           139 non-null    object 
dtypes: float64(9), object(1)
memory usage: 11.0+ KB


### Train Test Split and Data Preprocessing

In [7]:
X = df.drop('Region',1)
y = df['Region']

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=1)

In [9]:
y_train.value_counts()## checking whether all classes have proportional representation in our train data
## This step can be worked around with the stratify parameter (will be discussed in the next classes)

Sub-Saharan Africa            32
Europe & Central Asia         28
America                       22
East Asia & Pacific           10
Middle East & North Africa     8
South Asia                     4
Name: Region, dtype: int64

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [12]:
scaler = MinMaxScaler()## (X-max(x))/(max(X)-Min(X))

In [13]:
X_train = scaler.fit_transform(X_train)## Fit the scaling on the train dataset and return the scaled dataset in to the variables
X_test = scaler.transform(X_test)## Transform the test dataset with the scaler fitted on the train dataset

In [18]:
X

population  fertility   HIV        CO2  BMI_male      GDP  BMI_female  \
0    34811059.0       2.73   0.1   3.328945  24.59620  12314.0    129.9049   
1    19842251.0       6.43   2.0   1.474353  22.25083   7103.0    130.1247   
2    40381860.0       2.24   0.5   4.785170  27.50170  14646.0    118.8915   
3     2975029.0       1.40   0.1   1.804106  25.35542   7383.0    132.8108   
4    21370348.0       1.96   0.1  18.016313  27.56373  41312.0    117.3755   
..          ...        ...   ...        ...       ...      ...         ...   
134   3350832.0       2.11   0.5   2.489764  26.39123  15317.0    124.2604   
135  26952719.0       2.46   0.1   4.476669  25.32054   3733.0    124.3462   
136  86589342.0       1.86   0.4   1.479347  20.91630   4085.0    121.9367   
137  13114579.0       5.88  13.6   0.148982  20.68321   3039.0    132.4493   
138  13495462.0       3.85  15.1   0.654323  22.02660   1286.0    131.9745   

     life  child_mortality  
0    75.3             29.5  
1    58.3            192.0  
2    75.5             15.4  
3    72.5             20.0  
4    81.5              5.2  
..    ...              ...  
134  76.0             13.0  
135  68.7             49.2  
136  75.4             26.2  
137  52.0             94.9  
138  49.0             98.3  

[139 rows x 9 columns]

In [16]:
pd.DataFrame(X_train,columns = X.columns)

population  fertility       HIV       CO2  BMI_male       GDP  \
0      0.150400   0.018083  0.040248  0.144244  0.623653  0.065138   
1      0.003933   0.542495  0.202786  0.021993  0.456449  0.121223   
2      0.018987   0.650995  0.121517  0.004998  0.183798  0.005028   
3      0.018967   0.321881  0.009288  0.014170  0.638349  0.048578   
4      0.001548   0.264014  0.044118  0.042404  0.407867  0.036816   
..          ...        ...       ...       ...       ...       ...   
99     0.230335   0.157324  0.000000  0.082419  0.782461  0.126434   
100    0.000424   0.018083  0.001548  0.126796  0.904043  0.217423   
101    1.000000   0.142857  0.020898  0.380695  1.000000  0.396819   
102    0.009673   0.025316  0.001548  0.092207  0.801979  0.180376   
103    0.018826   0.188065  0.028638  0.021751  0.740746  0.054683   

     BMI_female      life  child_mortality  
0      0.777259  0.625000         0.053883  
1      0.714062  0.334239         0.344955  
2      0.750981  0.402174         0.494982  
3      0.337930  0.771739         0.121500  
4      0.423564  0.483696         0.207079  
..          ...       ...              ...  
99     0.363154  0.812500         0.103011  
100    0.368149  0.983696         0.020602  
101    0.051557  0.896739         0.026413  
102    0.739710  0.728261         0.029054  
103    0.132631  0.785326         0.099842  

[104 rows x 9 columns]

In [19]:
pd.DataFrame(X_train,columns = X.columns).describe()

population   fertility         HIV         CO2    BMI_male         GDP  \
count  104.000000  104.000000  104.000000  104.000000  104.000000  104.000000   
mean     0.084319    0.319151    0.079372    0.087611    0.522570    0.124687   
std      0.139875    0.274239    0.172785    0.134541    0.272313    0.154052   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.011019    0.106239    0.001548    0.009260    0.262474    0.018279   
50%      0.033335    0.211573    0.015093    0.034717    0.587536    0.063269   
75%      0.095471    0.530289    0.060565    0.124109    0.746236    0.166952   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

       BMI_female        life  child_mortality  
count  104.000000  104.000000       104.000000  
mean     0.508946    0.651468         0.233832  
std      0.255190    0.254339         0.239343  
min      0.000000    0.000000         0.000000  
25%      0.309061    0.457201         0.039884  
50%      0.496343    0.703804         0.122029  
75%      0.717078    0.852582         0.401479  
max      1.000000    1.000000         1.000000

### Model Building

In [13]:
clf = KNeighborsClassifier(n_neighbors = 5)#K = 5
## Other Hyper parameters : Weights(Based on distance),Distance Metric(metric)(Default is minkowsi[2])

In [14]:
clf.fit(X_train,y_train)

KNeighborsClassifier()

In [15]:
y_pred = clf.predict(X_test)

In [22]:
print(classification_report(y_pred,y_test))

                            precision    recall  f1-score   support

                   America       0.60      0.75      0.67         4
       East Asia & Pacific       0.00      0.00      0.00         0
     Europe & Central Asia       1.00      0.50      0.67        18
Middle East & North Africa       0.00      0.00      0.00         0
                South Asia       0.00      0.00      0.00         1
        Sub-Saharan Africa       0.86      1.00      0.92        12

                  accuracy                           0.69        35
                 macro avg       0.41      0.38      0.38        35
              weighted avg       0.88      0.69      0.74        35



C:\Users\yashm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yashm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yashm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yashm\anaconda3\lib\site-packages\sklearn\metrics\_cla